# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

import functools # for partial parametres

In [182]:
import gensim # td-idf

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from nltk.corpus import wordnet as wn # sentence similarity with wordnet

In [ ]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn

In [ ]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

# Пример сравнения похожести документов (Тупой, через TD-IDF статистики)

In [15]:
raw_documents = ["I'm taking the show on the road.",
                 "My socks are a force multiplier.",
             "I am the barber who cuts everyone's hair who doesn't cut their own.",
             "Legend has it that the mind is a mad monkey.",
            "I make my own fun."]
print("Number of documents:",len(raw_documents))

Number of documents: 5


In [16]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]
print(gen_docs)

[['i', "'m", 'taking', 'the', 'show', 'on', 'the', 'road', '.'], ['my', 'socks', 'are', 'a', 'force', 'multiplier', '.'], ['i', 'am', 'the', 'barber', 'who', 'cuts', 'everyone', "'s", 'hair', 'who', 'does', "n't", 'cut', 'their', 'own', '.'], ['legend', 'has', 'it', 'that', 'the', 'mind', 'is', 'a', 'mad', 'monkey', '.'], ['i', 'make', 'my', 'own', 'fun', '.']]


In [17]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary[5])
print(dictionary.token2id['road'])
print("Number of words in dictionary:",len(dictionary))
for i in range(len(dictionary)):
    print(i, dictionary[i])

show
4
Number of words in dictionary: 36
0 'm
1 .
2 i
3 on
4 road
5 show
6 taking
7 the
8 a
9 are
10 force
11 multiplier
12 my
13 socks
14 's
15 am
16 barber
17 cut
18 cuts
19 does
20 everyone
21 hair
22 n't
23 own
24 their
25 who
26 has
27 is
28 it
29 legend
30 mad
31 mind
32 monkey
33 that
34 fun
35 make


Now we will create a corpus. A corpus is a list of bags of words. A bag-of-words representation for a document just lists the number of times each word occurs in the document.

In [18]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2)], [(1, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)], [(1, 1), (2, 1), (7, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2)], [(1, 1), (7, 1), (8, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)], [(1, 1), (2, 1), (12, 1), (23, 1), (34, 1), (35, 1)]]


Что такое TF-IDF

http://nlpx.net/archives/57

In [19]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=5, num_nnz=47)
47


Now we will create a similarity measure object in tf-idf space.
tf-idf stands for term frequency-inverse document frequency. Term frequency is how often the word shows up in the document and inverse document fequency scales the value by how rare the word is in the corpus.

In [25]:
sims = gensim.similarities.Similarity('./workdir/',tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

Similarity index with 5 documents in 0 shards (stored under ./workdir/)
<class 'gensim.similarities.docsim.Similarity'>


Now create a query document and convert it to tf-idf.

In [26]:
query_doc = [w.lower() for w in word_tokenize("Socks are a force for good.")]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

['socks', 'are', 'a', 'force', 'for', 'good', '.']
[(1, 1), (8, 1), (9, 1), (10, 1), (13, 1)]
[(8, 0.31226270667960454), (9, 0.54848032538919966), (10, 0.54848032538919966), (13, 0.54848032538919966)]


We show an array of document similarities to query. We see that the second document is the most similar with the overlapping of socks and force.

# Схожесть предложений с использованием Wordnet

In [108]:
# You can check out how frequent a lemma is by doing:
cat = wn.synsets('cat', 'n')[0]     # Get the most common synset
print (cat.lemmas()[0].count()  )     # Get the first lemma => 18
 
dog = wn.synsets('dog', 'n')[0]           # Get the most common synset
feline = wn.synsets('feline', 'n')[0]     # Get the most common synset
mammal = wn.synsets('mammal', 'n')[0]     # Get the most common synset
 
# You can read more about the different types of wordnet similarity measures here: http://www.nltk.org/howto/wordnet.html
for synset in [dog, feline, mammal]:
    print ("Similarity(%s, %s) = %s" % (cat, synset, cat.wup_similarity(synset)))
 
# Similarity(Synset('cat.n.01'), Synset('dog.n.01')) = 0.2
# Similarity(Synset('cat.n.01'), Synset('feline.n.01')) = 0.5
# Similarity(Synset('cat.n.01'), Synset('mammal.n.01')) = 0.2
 

18
Similarity(Synset('cat.n.01'), Synset('dog.n.01')) = 0.8571428571428571
Similarity(Synset('cat.n.01'), Synset('feline.n.01')) = 0.9629629629629629
Similarity(Synset('cat.n.01'), Synset('mammal.n.01')) = 0.8333333333333334


http://nlpforhackers.io/wordnet-sentence-similarity/

In [ ]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 

In [131]:
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(sentence1)#pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(sentence2)#pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # -----------------------------------------------------------
    
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        curr_arr = [x for x in [synset.path_similarity(ss) for ss in synsets2] if x is not None]
        curr_arr.append(0)
#         print(curr_arr)
        
        best_score = max(curr_arr)
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1

    score /= count
    return score
 
sentences = [
    "Programming technologies",
    "Some gorgeous creatures are felines.",
    "Dolphins are swimming mammals.",
    "C++ Is beautiful shit.",
]
 
focus_sentence = "C++ are beautiful programming language"
 
for sentence in sentences:
    print("Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, sentence_similarity(focus_sentence, sentence)))
    print("Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, sentence_similarity(sentence, focus_sentence)))
 

Similarity("C++ are beautiful programming language", "Programming technologies") = 0.8608058608058607
Similarity("Programming technologies", "C++ are beautiful programming language") = 0.9103641456582633
Similarity("C++ are beautiful programming language", "Some gorgeous creatures are felines.") = 0.8068376068376069
Similarity("Some gorgeous creatures are felines.", "C++ are beautiful programming language") = 0.8219461697722565
Similarity("C++ are beautiful programming language", "Dolphins are swimming mammals.") = 0.6665503808360951
Similarity("Dolphins are swimming mammals.", "C++ are beautiful programming language") = 0.7090773809523809
Similarity("C++ are beautiful programming language", "C++ Is beautiful shit.") = 0.6167800453514738
Similarity("C++ Is beautiful shit.", "C++ are beautiful programming language") = 0.7896825396825397


# Технические теги

In [223]:
technical_tags = pd.read_csv('../DB/most_popular_stack_overflow.csv')
technical_tags = set(technical_tags['TagName'])
technical_tags = {x for x in technical_tags if x==x}

technical_tags_dict = {}
technical_tags_ind_dict = {}

index = 0
for x in technical_tags:
    technical_tags_dict[x] = index
    technical_tags_ind_dict[index] = x
    index += 1

# Синонимичный ряд замен для тегов технологий

In [224]:
technical_tags_syn = pd.read_csv('../DB/synonyms_for_stackoverflow.csv')
technical_tags_syn.head()

technical_tags_syn_dict = dict(zip(technical_tags_syn.SOurceTagName, technical_tags_syn.TargetTagName))
# technical_tags_syn.set_index('SOurceTagName')['TargetTagName'].to_dict('index')
technical_tags_syn_dict

def technical_tags_syn_try_replace(x):
    if x in technical_tags_syn_dict.keys():
        return technical_tags_syn_dict[x]
    else:
        return x

# Подобие семантической сети тегов технологий

In [225]:
small_stackoverflow = pd.read_csv('../DB/Small_stackoverflow.csv')
small_stackoverflow = small_stackoverflow['Tags']
small_stackoverflow.head()

0                  <html><css><layout><css-float>
1             <jquery><user-interface><usability>
2    <c#><.net><asynchronous><garbage-collection>
3                 <c#><asp.net><events><webforms>
4              <python><ajax><autocomplete><trie>
Name: Tags, dtype: object

In [226]:
semantic_technologies = pd.DataFrame(index=technical_tags,columns=technical_tags)
semantic_technologies = semantic_technologies.fillna(0)
semantic_technologies.head()

,entity-framework-5,mapbox,cakephp-2.0,apache-pig,dynamics-crm-2011,kibana,date,pyspark,xml-serialization,list-comprehension,...,query-string,microcontroller,testing,capybara,classpath,tooltip,android-6.0-marshmallow,haskell,autohotkey,converter
entity-framework-5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mapbox,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cakephp-2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
apache-pig,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dynamics-crm-2011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [247]:
for line in small_stackoverflow:
    currlist = line[1:-1].split('><')
    for x in currlist:
        for y in currlist:
            if ((x in technical_tags) and (y in technical_tags)):
                semantic_technologies.set_value(x, y, semantic_technologies.at[x, y] + 1)
                
semantic_tech_axis_sum0 = semantic_technologies.sum(axis=0)
semantic_tech_axis_sum1 = semantic_technologies.sum(axis=1)
# semantic_technologies.tail()
semantic_tech_axis_sum1.head()


for index, row in semantic_technologies.iterrows():
#     print(type(index))
    semantic_technologies.loc[index] = semantic_technologies.loc[index] /\
    (np.minimum(semantic_tech_axis_sum1[index], semantic_tech_axis_sum0))
    semantic_technologies.loc[index, index] = -1.0
semantic_technologies.fillna(0,inplace=True)

for x in technical_tags:
    print('For', x, 'popular tag is', np.argmax(semantic_technologies[x]))
# np.argmax(semantic_technologies['html'])
# semantic_technologies.loc['html', 'positioning']

# print(sorted(list(semantic_technologies['html'])))

For entity-framework-5 popular tag is c#
For mapbox popular tag is javascript
For cakephp-2.0 popular tag is php
For apache-pig popular tag is user-defined-functions
For dynamics-crm-2011 popular tag is javascript
For kibana popular tag is entity-framework-5
For date popular tag is java
For pyspark popular tag is rdd
For xml-serialization popular tag is c#
For list-comprehension popular tag is python
For reporting popular tag is php
For electron popular tag is javascript
For rdd popular tag is pyspark
For generator popular tag is python
For jquery-mobile popular tag is jquery
For jquery-select2 popular tag is jquery
For mod-wsgi popular tag is python
For server popular tag is php
For mapkit popular tag is ios
For cuda popular tag is c++
For latex popular tag is python
For build.gradle popular tag is android
For 3d popular tag is c#
For radio-button popular tag is android
For connection-pooling popular tag is java
For compatibility popular tag is jquery
For copy-paste popular tag is c#


In [211]:
np.maximum(1, [0, 1, 2])

array([1, 1, 2])

# Кастомный датасет

In [63]:
custom_small_set = pd.read_csv('small_custom_dataset.csv', sep=';')
custom_small_set

,id,name,geo,ementoring,sex,user_role,skills_tags,intro_phrase_tags
0,1,nn_mentor,100,0,male,mentor,"c++,java,selling,IT","teaching,freelance,advices,teamwork"
1,2,nn_mentor,200,0,female,mentor,"javascript,modelling,project,architecture","looking,students,control,progress"
2,3,pers_mentorMathHelper,120,0,male,mentor,"math,statistics,machine,learning,teaching,geom...","school,university,students,help,assistance,exams"
3,19,pers_mentorMathHelper2,5000,2,male,mentor,"math,analysis,optimization,logics,problems,dif...","advanced,skills,improve,difficulties,track,pro..."
4,4,pers_mentorManager,10000,0,male,mentor,"HR,management,analytics,business,stuff,IT","team,building,corporative,mentor,ideas,business"
5,5,pers_mentorDesktopDeveloper,7000,0,female,mentor,"c#,application,signal,processing,visualization...","assistance,track,progress,certain,problem,sett..."
6,9,pers_mentorDesktopDeveloperFarAway,170000,0,female,mentor,"c#,application,signal,processing,visualization...","assistance,track,progress,certain,problem,sett..."
7,6,pers_mentorWebGuy,5000,1,male,mentor,"web,design,imposition,layout,css,html,javascri...","study,no,self,forward,advice,direction"
8,7,pers_mentorArchitector,-666,1,female,mentor,"architecture,programming,planning,building,bac...","student,group,online,experience,building,OOP,i..."
9,8,pers_studentLookingForDesktop,6500,0,male,student,"nothing,Pascal,courses,algebra,geometry","desktop,applications,science,models"


# Общая структура происходящего

In [ ]:
data = custom_small_set

#### TODO

In [ ]:
def geo_distance_metrics(geo1, geo2):
       return abs(geo1 - geo2)

In [ ]:
def geo_close_enough(geo1, geo2):
    return geo_distance_metrics(geo1, geo2) < 3000

In [118]:
def closest_people(index):
    person = data.loc[index]
    
    print("Out req person:", person)
    
    stud_lambd1 = lambda x: \
(geo_close_enough(person.geo, x.geo) or (person.ementoring >= 1 and x.ementoring >= 1)) and \
x.user_role == 'mentor'

    ment_lambd1 = lambda x: \
(geo_close_enough(person.geo, x.geo) or (person.ementoring >= 1 and x.ementoring >= 1)) and \
x.user_role == 'student'
    
    if person.user_role == 'student':
        closest_people = data[data.apply(stud_lambd1, axis=1)]
        return closest_people
    elif person.user_role == 'mentor':
        closest_people = data[data.apply(ment_lambd1, axis=1)]
        return closest_people
        
#     return data.loc[index]

#### Жесткое косинусное расстояние

In [ ]:
def get_cosine(vec1, vec2):
    print('vec', vec1, vec2)
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

def list_to_vector(text):
    return Counter(text)

#### Мягкое косинусное расстояние

In [ ]:
def tech_tag_sim(w1, w2):
    if (w1 == w2):
        return 1.0
    elif (w1 in technical_tags and w2 in technical_tags):
        return semantic_technologies.loc[w1, w2]
    else:
        return 0.0
        

In [ ]:
def get_cosine_soft(vec1, vec2):
    print('vec', vec1, vec2)
    
    S1 = 0
    S2 = 0
    S3 = 0
    for x in vec1:
        for y in vec2:
            S1 += tech_tag_sim(x, y)
    for x in vec1:
        for y in vec1:
            S2 += tech_tag_sim(x, y)
    for x in vec2:
        for y in vec2:
            S3 += tech_tag_sim(x, y)
    
    S2 = S2**0.5
    S3 = S3**0.5
    
    if (S2 * S3 > 0):
        return S1 / (S2 * S3)
    else:
        return 0.0
#     intersection = set(vec1.keys()) & set(vec2.keys())
    
    
#     numerator = sum([vec1[x] * vec2[x] for x in intersection])

#     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
#     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
#     denominator = math.sqrt(sum1) * math.sqrt(sum2)

#     if not denominator:
#         return 0.0
#     else:
#         return float(numerator) / denominator

In [ ]:
get_cosine_soft(['c++'], ['c++'])

In [157]:
get_cosine_soft(['html'], ['css'])

In [ ]:
get_cosine_soft(['c++'], ['animals'])

https://stackoverflow.com/questions/15173225/calculate-cosine-similarity-given-2-sentence-strings

In [ ]:
def check_person(person, x):
    w0 = 0
    w1 = 1
    xskills = [technical_tags_syn_try_replace(x) for x in x.skills_tags.split(',')] #x.skills_tags.replace(",", " ")
    xintro = x.intro_phrase_tags.split(',') # x.intro_phrase_tags.replace(",", " ")#
    
    personskills = [technical_tags_syn_try_replace(x) for x in person.skills_tags.split(',')] #person.skills_tags.replace(",", " ")
    personintro = person.intro_phrase_tags.split(',')#person.intro_phrase_tags.replace(",", " ")#
    
    x_alltags = xskills + xintro
    person_alltags = personskills + personintro
    
    xtechnical = [x for x in x_alltags if x in technical_tags]
    persontechnical = [x for x in person_alltags if x in technical_tags]
    xhuman = [x for x in x_alltags if not (x in technical_tags)]
    personhuman = [x for x in person_alltags if (not x in technical_tags)]

    return w0 * sentence_similarity(xhuman, personhuman) +\
        w1 * get_cosine_soft(list_to_vector(xtechnical), list_to_vector(persontechnical))

In [ ]:
def process_req(index):
    person = data.loc[index]
    
    closest1 = closest_people(index)
    
    sLength = len(closest1['id'])
    closest1['tag_score'] = closest1.apply(functools.partial(check_person, person), axis=1)
    return closest1.sort_values(by=['tag_score'], ascending=False)

In [ ]:
process_req(2)

# Эксперименты

In [151]:
get_cosine(list_to_vector(['c++','java']), list_to_vector(['java','c#']))

0.4999999999999999

In [ ]:
import numpy as np

In [ ]:
np.array([(1, 2),(1, 2),(1, 2)]).shape

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame({'fieldA' : ['lol', 'kek', 'fuf'], 'fieldB' : ['anus1', 'anus2', 'anus1']})

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame({'fieldB' : ['anus1', 'anus2'], 'fieldC' : [100, 200]})
df2

In [ ]:
pd.merge(df1, df2, on='fieldB', how='inner')

In [ ]:
df2 = df2.rename(index=str, columns={"fieldB": "whore"})
df2

In [ ]:
nltk.download()